# **此程式為螞蟻演算法(ACO)應用實作**
----------------------

### **問題:在所有已知地點中，如何求出拜訪的最短路線? (地點不可重複拜訪)**

In [1]:
import sys
import math
import time
import random

### **1.定義相關參數**

In [2]:
alpha=1
beta=5
Q=1
E=8

#螞蟻數量
size=20

#城市數量
num=5

#城市間的距離(事先定義好)
distance= [[0.0, 100.0, 150.0, 500.0, 210.0], 
           [100.0, 0.0, 90.0, 100.0, 150.0], 
           [150.0, 90.0, 0.0, 40.0, 100.0], 
           [500.0, 100.0, 40.0, 0.0, 130.0], 
           [210.0, 150.0, 100.0, 130.0, 0.0]]


### **2.初始化**

In [3]:
def _initial_(size, num, a, b, city_distance):

    #可見度
    global Visibility
    
    #費洛蒙
    global Pheromone
    
    #螞蟻所在位置(城市)
    global ants_address
    
    #儲存路線的list
    global t
    
    global prate
    
    #全域(歷史)最佳路徑
    global GBestTour
    
    #全域(歷史)最佳路徑之距離
    global GBestFitness
    
    Visibility=[[0.0]*num]*num
    Pheromone=[[0.0]*num]*num
    ants_address=[0]*size
    t=[0]*num
    prate=[0.0]*num
    GBestTour=[0]*num
    GBestFitness=sys.float_info.max 

    for i in range(num):
        for j in range(num):
            distance[i][j] = city_distance[i][j]
            if (i != j):
                #初始費洛蒙
                Pheromone[i][j]=1.0
                #初始可見度
                Visibility[i][j]=1/distance[i][j] 
    #初始螞蟻位置
    for i in range(size):
        ants_address[i] = random.randint(0, num-1)

### **3.行進。 根據「費洛蒙」跟「可見度訊息」選擇下一個要前往的城市**

In [4]:
def walk(where_ant):
    for i in range(num):
        t[i] = i
        
    # t[0] 為當下初始的點
    tmp=t[0]
    t[0]=t[where_ant]
    t[where_ant]=tmp

    #做剩下 n - 1 次
    for i in range(1,num):
        
        nowrate=[0]*num
        nowrate_sum=0.0
        
        #可選擇的點個別機率 & 總和
        for j in range(i,num):
            k=i-1
            #p(i) 分母
            nowrate_sum += (math.pow(Pheromone[t[k]][t[j]], alpha) * math.pow(Visibility[t[k]][t[j]], beta))

        for j in range(i,num):
            k=i-1
            #p(i) value && 正規化
            nowrate[j] = (math.pow(Pheromone[t[k]][t[j]], alpha) * math.pow(Visibility[t[k]][t[j]], beta)) / nowrate_sum
        
        #走輪盤法
        r = random.random()
        now=i
        while (r > 0):
            r -= nowrate[now]
            if(r > 0):
                now+=1

        #i值位置前 表示 已經走過的城市(過程按i值)，位置後 表示 還未到的城市
        tmp=t[i]
        t[i]=t[now]
        t[now]=tmp

### **4.迭代。 根據每一隻螞蟻選擇的路線，計算分數，並讓費洛蒙揮發**

In [5]:
def iter(MaxCycle):

    currentCycle = 1
    global GBestFitness
    
    while (currentCycle <= MaxCycle):
        
        for j in range(len(ants_address)): 
            
            #走滿一次cycle
            walk(ants_address[j])
            
            #得到路線總距離
            tour = Fitness(t)
            
            #判斷路線是否好
            if tour < GBestFitness:
                GBestFitness = tour
                for i in range(num):
                    GBestTour[i] = t[i]
        #Pheromone揮發
        for i in range(num - 1):
            for k in range(num):
                Pheromone[i][k] = Pheromone[i][k] * 0.99

        #對目前最好的路線重點標記一次
        for i in range(num - 1):
            Pheromone[GBestTour[i]][GBestTour[i + 1]] += (E * Q / GBestFitness)
        Pheromone[GBestTour[num - 1]][GBestTour[0]] += (E * Q / GBestFitness)
        
        currentCycle+=1

### **5.定義Fitness。 預設為路線所產生的距離總和**

In [6]:
def Fitness(t):
    sum=0
    for i in range(len(t)-1):
        sum+=distance[t[i+1]][t[i]]
    return sum

### **6.訓練**

In [7]:
def start(run_length,FunctionEvaluationCount):
    
    best_fitness=[0.0]*run_length
    bf_sum=0.0
    
    for run in range(run_length):
    
        _initial_(size, num, alpha, beta, distance)
        iter(FunctionEvaluationCount)
    
        best_fitness[run]=GBestFitness
        bf_sum+=best_fitness[run]
    
        print(f"No.{run + 1} Run: Best Fitness = {best_fitness[run]}")
    print("最佳路線為:",GBestTour)

### **7.結果**
#### **\#城市間距離是自定義的，不難，很快便可求出**
#### **\#最佳路線可能不只一個**

In [8]:
start(3,100)

No.1 Run: Best Fitness = 340.0
No.2 Run: Best Fitness = 340.0
No.3 Run: Best Fitness = 340.0
最佳路線為: [4, 2, 3, 1, 0]


-----------------------
#### **可延伸出其他應用，例如地點1跟地點3有重要客戶，需要優先拜訪**
#### **此時修改fitness計算方式如下:**

In [9]:
def Fitness(t):
    sum=0
    for i in range(len(t)-1):
        
        #越早拜訪1跟3，分數越好
        if(t[i]==0 or t[i]==2):
            sum+=i**2*1000
        sum+=distance[t[i+1]][t[i]]

    #補迴圈漏掉的部分
    if t[len(t)-1]==0 or t[len(t)-1]==2 :
        sum+=len(t)**2*1000
    return sum

#### **結果如下:**

In [10]:
start(3,100)

No.1 Run: Best Fitness = 1440.0
No.2 Run: Best Fitness = 1440.0
No.3 Run: Best Fitness = 1440.0
最佳路線為: [0, 2, 3, 1, 4]
